In [91]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score

## 1 SVM with Submanifold Minimization

In [92]:
class SVM:
    def __init__(self):
        pass

    def learn_svm(self, X, y):

        N = len(y)

        C = np.outer(y, y) * np.dot(X, X.T)  
        c = -np.ones(N)  
        A = y.reshape(1, -1)  
        a = np.array([0]) 
        B = -np.eye(N)  
        b = np.zeros(N)  

        N_plus = np.sum(y == +1)  
        N_minus = N - N_plus  
        alpha_0 = np.array([1 / N_plus if label == +1 else 1 / N_minus for label in y])

        alpha = self.min_ineq_cstr_subm(C, c, A, a, B, b, alpha_0)

        beta = np.sum((alpha * y).reshape(-1, 1) * X, axis=0)

        support_indices = np.where(alpha > 1e-6)[0]  # Support vectors: alpha_n > 0
        beta_0 = np.mean([y[n] - np.dot(beta, X[n]) for n in support_indices])

        return beta_0, beta

    def predict(self, X, beta_0, beta):
        
        return np.sign(np.dot(X, beta) + beta_0)

    def min_ineq_cstr_subm(self, C, c, A, a, B, b, x):
    
        K0 = {k for k in range(len(b)) if np.dot(B[k], x) - b[k] == 0}

        max_iter = 1000
        iteration = 0
        while True:
            while True:
                A_tilde = np.vstack([A, B[list(K0), :]])
                a_tilde = np.concatenate([a, b[list(K0)]])

                ''' Matrix consisting of different blocks 
                    [C        A_tilde_transposed]
                    [A_tilde                   0]
                '''
                caa0 = np.block([
                    [C, A_tilde.T],
                    [A_tilde, np.zeros((A_tilde.shape[0], A_tilde.shape[0]))]
                ])
                ca = np.concatenate([c, a_tilde])

                s = np.dot(np.linalg.pinv(caa0), ca)  # Use pseudoinverse in case the matrix is singular

                x_star = s[:C.shape[0]]
                v_star = s[C.shape[0]:]

                f_x_star = 0.5 * np.dot(x_star.T, np.dot(C, x_star)) - np.dot(c, x_star)
                f_x = 0.5 * np.dot(x.T, np.dot(C, x)) - np.dot(c, x)

                if f_x_star >= f_x:
                    break

                mu = 1.0
                while np.any(np.dot(B, x + mu * (x_star - x)) - b > 0):
                    mu *= 0.8
                    if mu < 1e-8: 
                        return x

                x = x + mu * (x_star - x)
                K0 = {k for k in range(len(b)) if np.dot(B[k], x) - b[k] == 0}

            if np.all(v_star >= 0):
                break

            for k in K0:
                if v_star[k] < 0:
                    K0.remove(k)
                    break

            iteration += 1
            if iteration == max_iter:
                print("Max iterations")
                break

        return x


In [93]:
Xpos = np.array([
    [2.0, 2.2],
    [2.7, 2.5],
    [2.3, 2.0],
    [3.1, 2.3],
    [2.5, 2.4],
    [2.8, 2.7],
])
ypos = np.ones(len(Xpos))

Xneg = np.array([
    [1.6, 1.5],
    [2.0, 1.9],
    [2.1, 1.8],
    [1.7, 1.6],
    [1.8, 1.7],
    [2.0, 1.6],
])
yneg = -np.ones(len(Xneg))

X = np.vstack([Xpos, Xneg])  
y = np.hstack([ypos, yneg])  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

svm_classifier = SVM()

beta_0, beta = svm_classifier.learn_svm(X_train, y_train)

y_pred = svm_classifier.predict(X_test, beta_0, beta)

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Recall: {recall:.4f}")

Accuracy: 0.7500
F1 Score: 0.6667
Recall: 1.0000
